### 저장 된 모델 활용
- 모델 파일 종류
    - 가중치 및 절편 저장 파일 => 동일한 구조 모델 인스턴스 생성 후 사용 가능
    - 모델 전체 저장 파일 => 바로 로딩 후 사용 가능


[1] 모듈로딩 <hr>

In [1]:
# 모듈로딩
# Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary


In [2]:
class IrisMCFModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(4,10)
        self.hd_layer=nn.Linear(10,5)
        self.out_layer=nn.Linear(5,3)   # 다중분류

    # 순방향 학습 진행 메서드
    def forward(self,x):
        # - 입력층
        y=self.in_layer(x)        # f1_1w1_1+f1_2w1_2+f1_3w1_3+b << 이런식 10개
        y=F.relu(y)

        # - 은닉층 : 10개의 숫자값 (>=0)
        y=self.hd_layer(y)        # f2_1w2_1+f2_2w2_2+....f2_10w2_10 +b << 이런식 5개
        y=F.relu(y)

        # - 출력층 : 5개의 숫자값 (>=0) - 다중분류 : 손실함수 crossentroploss 가 내부에서 softmax진행
        return self.out_layer(y)        # f3_1w3_1+.....f3_30w3_30+b << 1 개




In [3]:
import torch
import torch.nn as nn 

In [4]:
## 모델 파일 관련
### models 폴더 아래 프로젝트 폴더 아래 모델파일 저장
import os

# 저장경로
SAVE_PATH='../models/iris/MCF/'
# 저장 파일명
SAVE_FILE=SAVE_PATH+'model_train_wbs.pth'

# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL=SAVE_PATH+'model_all.pth'



[2] 모델 로딩 - 모델 전체 파일 사용<hr>

In [5]:
if os.path.exists(SAVE_MODEL):
    irisModel=torch.load(SAVE_MODEL,weights_only=False)
else:
    print(f'{SAVE_MODEL}이 존재하지 않습니다.')

In [6]:
summary(irisModel)

Layer (type:depth-idx)                   Param #
IrisMCFModel                             --
├─Linear: 1-1                            50
├─Linear: 1-2                            55
├─Linear: 1-3                            18
Total params: 123
Trainable params: 123
Non-trainable params: 0

[3] 예측<hr>

In [7]:
iris=[float(x) for x in input('SL, SW, PL, PW:').split(',')]
iris

ValueError: could not convert string to float: ''

In [78]:
dataTS=torch.FloatTensor(iris).reshape(1,-1)
dataTS

tensor([[1.3300, 2.8300, 5.3200, 3.4200]])

In [81]:
# 새로운 데이터에 대한 예측 즉, predict
irisModel.eval() # 검증모드 켜기!
with torch.no_grad():
    # 추론/평가
    pre_val=irisModel(dataTS)

print(pre_val)

tensor([[-0.3723,  0.2482,  0.3948]])


In [80]:
iris={'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}
for i,num in enumerate(pre_val[0]):
    if num==pre_val[0].max():
        print(list(iris.keys())[i])

Virginica
